# Data Cleaning and initial EDA Workbook for Project 2

---

The purpose of this workbook is to load the raw data for the project and to clean variables.  It is set up so that if new variables need attention, it starts from the raw data and produces all the changes from the beginning.  In this way, it is clear that our source is the initial raw file.  

I flipped between this workbook and another, more EDA focused workbook.  As EDA discovered items to look at, features to build, etc, I modified the datafiles accordingly.  

At the end of the notebook, csv files are generated so that other notebooks can pick up the cleaned data and use them as required.  

Input:  test.csv and train.csv  
Output:  test_nonulls#.csv and train_nonulls#.csv

The variable `out_numb` should be updated each time a new data file is generated.

---

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math

pd.options.display.max_columns = 999             # Allows viewing of more rows and columns in a dataframe
pd.options.display.max_rows = 999

% matplotlib inline

In [2]:
testfile = './datasets/test.csv'
df_test = pd.read_csv(testfile)

trainfile = './datasets/train.csv'
df = pd.read_csv(trainfile)

out_numb = '7'
outfile_train = './datasets/train_nonulls' + out_numb + '.csv'
outfile_test = './datasets/test_nonulls' + out_numb + '.csv'

#sample_out = './sample_sub_reg.csv'
#df_out = pd.read_csv(sample_out)

#print(len(df), len(df_test), len(df_out))

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,2051.0,1.474034e+03,8.439808e+02,1.0,753.5,1486.0,2.198000e+03,2930.0
PID,2051.0,7.135900e+08,1.886918e+08,526301100.0,528458140.0,535453200.0,9.071801e+08,924152030.0
MS SubClass,2051.0,5.700878e+01,4.282422e+01,20.0,20.0,50.0,7.000000e+01,190.0
Lot Frontage,1721.0,6.905520e+01,2.326065e+01,21.0,58.0,68.0,8.000000e+01,313.0
Lot Area,2051.0,1.006521e+04,6.742489e+03,1300.0,7500.0,9430.0,1.151350e+04,159000.0
Overall Qual,2051.0,6.112140e+00,1.426271e+00,1.0,5.0,6.0,7.000000e+00,10.0
Overall Cond,2051.0,5.562165e+00,1.104497e+00,1.0,5.0,5.0,6.000000e+00,9.0
Year Built,2051.0,1.971709e+03,3.017789e+01,1872.0,1953.5,1974.0,2.001000e+03,2010.0
Year Remod/Add,2051.0,1.984190e+03,2.103625e+01,1950.0,1964.5,1993.0,2.004000e+03,2010.0
Mas Vnr Area,2029.0,9.969591e+01,1.749631e+02,0.0,0.0,0.0,1.610000e+02,1600.0


In [4]:
df.isnull().sum()

Id                    0
PID                   0
MS SubClass           0
MS Zoning             0
Lot Frontage        330
Lot Area              0
Street                0
Alley              1911
Lot Shape             0
Land Contour          0
Utilities             0
Lot Config            0
Land Slope            0
Neighborhood          0
Condition 1           0
Condition 2           0
Bldg Type             0
House Style           0
Overall Qual          0
Overall Cond          0
Year Built            0
Year Remod/Add        0
Roof Style            0
Roof Matl             0
Exterior 1st          0
Exterior 2nd          0
Mas Vnr Type         22
Mas Vnr Area         22
Exter Qual            0
Exter Cond            0
Foundation            0
Bsmt Qual            55
Bsmt Cond            55
Bsmt Exposure        58
BsmtFin Type 1       55
BsmtFin SF 1          1
BsmtFin Type 2       56
BsmtFin SF 2          1
Bsmt Unf SF           1
Total Bsmt SF         1
Heating               0
Heating QC      

#### Lot Frontage 
Lot frontage should not be 0.  To estimate a value for lot frontage when missing, I am assuming that a relationship exists between the amount of 'Lot Frontage' and the 'Lot Area'.  Below, I find the mean of the various ratios of Lot Frontage to Lot Area.  I then apply that average to the Lot Area in the case that Lot Frontage is not available.

There were two different ways to estimatet this amount.  I used 0.0079 as the estimate and sensitivity tested using 0.0071.  There was not a material difference in my results based on the sensitivity test.  

In [5]:
print(df[df['Lot Frontage'].notnull()]['Lot Frontage'].sum() / (df[df['Lot Frontage'].notnull()]['Lot Area'].sum()))
print(np.mean(df[df['Lot Frontage'].notnull()]['Lot Frontage'] / df[df['Lot Frontage'].notnull()]['Lot Area']))

df.loc[df['Lot Frontage'].isnull(), 'Lot Frontage'] = df[df['Lot Frontage'].isnull()]['Lot Area']*0.0076
df_test.loc[df_test['Lot Frontage'].isnull(), 'Lot Frontage'] = df_test[df_test['Lot Frontage'].isnull()]['Lot Area']*0.0076

0.007176381971631903
0.007918384398949624


#### Alley

The data dictionary allows NA as a response indicating no alley -- change this to a more descriptive NoAlley

In [6]:
df.loc[df['Alley'].isnull(), 'Alley'] = 'NoAlley'
df_test.loc[df_test['Alley'].isnull(), 'Alley'] = 'NoAlley'


#### Masonry Type and Area

The data dictionary allows None as a response indicating no masonry -- change this to a more descriptive NoMasonryType and 0 (for Mas Vnr Area).

In [7]:
df.loc[df['Mas Vnr Type'].isnull(), 'Mas Vnr Type'] = 'NoMasonryType'
df.loc[df['Mas Vnr Area'].isnull(), 'Mas Vnr Area'] = 0
df_test.loc[df_test['Mas Vnr Type'].isnull(), 'Mas Vnr Type'] = 'NoMasonryType'
df_test.loc[df_test['Mas Vnr Area'].isnull(), 'Mas Vnr Area'] = 0


#### Basement fields.  

Note first need to record no basement values and for exposure and type 2.  There will be some additional work after that. Again, n/a or none was a valid entry that caused python to interpret this as a null.  For numerical values, I assumed 0 and for other cases, I created a 'NoB' identifier meaning 'No Basement'

Bsmt Qual            55  
Bsmt Cond            55  
Bsmt Exposure        58  
BsmtFin Type 1       55  
BsmtFin SF 1          1  
BsmtFin Type 2       56  
BsmtFin SF 2          1  
Bsmt Unf SF           1  
Total Bsmt SF         1  
Bsmt Full Bath        2  
Bsmt Half Bath        2  

Assumed that if a BsmtFin Type 2 is null, that it could use the value in BsmtFin Type 1.

In [8]:
df.loc[df['Bsmt Qual'].isnull(), ['Bsmt Qual', 
                                  'Bsmt Cond', 
                                  'Bsmt Exposure', 
                                  'BsmtFin Type 1',
                                  'BsmtFin Type 2']] = 'NoB'     # No basement

df.loc[df['Bsmt Exposure'].isnull(), 'Bsmt Exposure'] = 'NoEx'
df.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 2'] = df.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 1']
df.loc[df['BsmtFin SF 1'].isnull(), 'BsmtFin SF 1'] = 0
df.loc[df['BsmtFin SF 2'].isnull(), 'BsmtFin SF 2'] = 0
df.loc[df['Bsmt Unf SF'].isnull(), 'Bsmt Unf SF'] = 0
df.loc[df['Total Bsmt SF'].isnull(), 'Total Bsmt SF'] = 0


df_test.loc[df_test['Bsmt Qual'].isnull(), ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1','BsmtFin Type 2']] = 'NoB'
df_test.loc[df_test['Bsmt Exposure'].isnull(), 'Bsmt Exposure'] = 'NoEx'
df_test.loc[df_test['BsmtFin Type 2'].isnull(), 'BsmtFin Type 2'] = df_test.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 1']
df_test.loc[df_test['BsmtFin SF 1'].isnull(), 'BsmtFin SF 1'] = 0
df_test.loc[df_test['BsmtFin SF 2'].isnull(), 'BsmtFin SF 2'] = 0
df_test.loc[df_test['Bsmt Unf SF'].isnull(), 'Bsmt Unf SF'] = 0
df_test.loc[df_test['Total Bsmt SF'].isnull(), 'Total Bsmt SF'] = 0

df.loc[df['Bsmt Full Bath'].isnull(), 'Bsmt Full Bath'] = 0
df.loc[df['Bsmt Half Bath'].isnull(), 'Bsmt Half Bath'] = 0

df_test.loc[df['Bsmt Full Bath'].isnull(), 'Bsmt Full Bath'] = 0
df_test.loc[df['Bsmt Half Bath'].isnull(), 'Bsmt Half Bath'] = 0

#### Fireplace Quality 

The data dictionary allows an n/a for fireplace quality.  

I created a new value 'NoF' for 'No Fireplace' in these situations.   



In [9]:
df.loc[df['Fireplace Qu'].isnull(), 'Fireplace Qu'] = 'NoF'
df_test.loc[df_test['Fireplace Qu'].isnull(), 'Fireplace Qu'] = 'NoF'

#### Garage variables

The data dictionary allows N/A for no garage  

For most, the values should be set to none.  I used 'NoGarage' for text columns and 0 for numerical columns.  
For the item that's left, I imputed values based on averages / most prevalent

- Garage Type         113
- Garage Yr Blt       114
- Garage Finish       114
- Garage Cars           1
- Garage Area           1
- Garage Qual         114
- Garage Cond         114

In [10]:
df.loc[df['Garage Type'].isnull(), ['Garage Yr Blt']] = 0
df.loc[df['Garage Type'].isnull(), ['Garage Type', 'Garage Finish', 'Garage Qual','Garage Cond']] = 'NoGarage'

df_test.loc[df['Garage Type'].isnull(), ['Garage Yr Blt']] = 0       
df_test.loc[df_test['Garage Type'].isnull(), ['Garage Type', 'Garage Finish', 'Garage Qual','Garage Cond']] = 'NoGarage'

--- 

For one record, there are not values even though (through EDA), there was a detached garage.  It doesn't really matter what we put here since it's only one record, I decided to query the other records and put in the mean value of the population.  In this case, year build = 1962, cars = 2 and square footage = 420.  Since there is a garage but it is not recorded, I assumed "low quality" metrics for the text variables (unfinished and TA).

---


In [11]:
print(df[df['Garage Type']=='Detchd']['Garage Yr Blt'].mean())
print(df[df['Garage Type']=='Detchd']['Garage Cars'].mean())
print(df[df['Garage Type']=='Detchd']['Garage Area'].mean())
#round these estimates and use to fill in missing data

1962.2616822429907
1.5588785046728972
419.54392523364487


In [12]:
df.loc[df['Garage Yr Blt'].isnull(), 'Garage Yr Blt'] = 1962
df.loc[df['Garage Finish'].isnull(), 'Garage Finish'] = 'Unf'
df.loc[df['Garage Cars'].isnull(), 'Garage Cars'] = 2
df.loc[df['Garage Area'].isnull(), 'Garage Area'] = 420
df.loc[df['Garage Qual'].isnull(), 'Garage Qual'] = 'TA'
df.loc[df['Garage Cond'].isnull(), 'Garage Cond'] = 'TA'


df_test.loc[df_test['Garage Yr Blt'].isnull(), 'Garage Yr Blt'] = 1962
df_test.loc[df_test['Garage Finish'].isnull(), 'Garage Finish'] = 'Unf'
df_test.loc[df_test['Garage Cars'].isnull(), 'Garage Cars'] = 2
df_test.loc[df_test['Garage Area'].isnull(), 'Garage Area'] = 420
df_test.loc[df_test['Garage Qual'].isnull(), 'Garage Qual'] = 'TA'
df_test.loc[df_test['Garage Cond'].isnull(), 'Garage Cond'] = 'TA'

#### Remaining null values

- Pool QC            2042
- Fence              1651
- Misc Feature       1986
- Electrical            1

These each have N/A as an option and they were all text.  I used a new category indicating "NoXYZ" instead of the data dictionary N/A.

In [13]:
df.loc[df['Pool QC'].isnull(), 'Pool QC'] = 'NoPool'
df.loc[df['Fence'].isnull(), 'Fence'] = 'NoFence'
df.loc[df['Misc Feature'].isnull(), 'Misc Feature'] = 'NoMiscF'

df_test.loc[df_test['Pool QC'].isnull(), 'Pool QC'] = 'NoPool'
df_test.loc[df_test['Fence'].isnull(), 'Fence'] = 'NoFence'
df_test.loc[df_test['Misc Feature'].isnull(), 'Misc Feature'] = 'NoMiscF'
df_test.loc[df_test['Electrical'].isnull(), 'Electrical'] = 'SBrkr'

#### New Features

---

I thought it might be instructive to use the maximum of the year built, the year remodeled, or garage added as a potential indication of when substantive improvements to the property were enacted.  New varialbe 'Yr Latest Change' added to the data frame.

---


In [14]:
df['Yr Latest Change'] = df[['Garage Yr Blt','Year Built', 'Year Remod/Add', ]].max(axis = 1)
df_test['Yr Latest Change'] = df_test[['Garage Yr Blt','Year Built', 'Year Remod/Add', ]].max(axis = 1)

--- 

The neighborhood variable is likely not usefull with as many neighborhoods involved.  I decided to group the neighborhoods by average sales price to create new categories.

---

In [15]:
df['Neighborhood'].value_counts()

NAmes      310
CollgCr    180
OldTown    163
Edwards    143
Somerst    130
NridgHt    122
Gilbert    116
Sawyer     111
NWAmes      87
SawyerW     87
Mitchel     82
BrkSide     76
Crawfor     71
IDOTRR      69
Timber      48
NoRidge     48
StoneBr     38
SWISU       32
ClearCr     27
MeadowV     24
Blmngtn     22
BrDale      19
NPkVill     17
Veenker     17
Blueste      6
Greens       3
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [16]:
def get_neigh_cat(list_in):
    list_out = []
    cat1 = ['StoneBr','NridgHt','NoRidge']
    cat2 = ['GrnHill','Veenker','Timber']
    cat3 = ['Somerst','ClearCr','Crawfor','CollgCr', 'Blmngtn']
    cat4 = ['NWAmes','Gilbert','Greens','SawyerW']
    cat5 = ['Mitchel','NAmes','Blueste']
    cat6 = ['NPkVill','Sawyer','Landmrk','SWISU','Edwards','BrkSide', 'OldTown']
    cat7 = ['BrDale','IDOTRR','MeadowV']
    for each in list_in:
        if each in cat1:
            list_out.append('cat1')
        elif each in cat2:
            list_out.append('cat2')
        elif each in cat3:
            list_out.append('cat3')
        elif each in cat4:
            list_out.append('cat4')
        elif each in cat5:
            list_out.append('cat5')
        elif each in cat6:
            list_out.append('cat6')
        elif each in cat7:
            list_out.append('cat7')
        else:
            list_out.append(each)
    return(list_out)
            

In [17]:
df['Neighb feat'] = get_neigh_cat(df['Neighborhood'])
df_test['Neighb feat'] = get_neigh_cat(df_test['Neighborhood'])

---

For this iteration of data files, I have decided to take the log of certain items that are skewed right.  Lot frontage, lot area, gr living area, and total bsmt SF.  In the modeling notebook, I will make two other changes, MS SubClass turned into category variable, and exclude other SF variables.  For log function, need the python function that allows log(0)

---

In [18]:
var_list = ['Lot Frontage', 'Lot Area', 'Gr Liv Area', 'Total Bsmt SF']

for each in var_list:
    df[each] = np.log1p(df[each])
    df_test[each] = np.log1p(df[each])


In [19]:
print(sum(df.isnull().sum()))
print(sum(df_test.isnull().sum()))

#df_test.isnull().sum()

0
0


## Output cleansed data into new data file

In [20]:
df.to_csv(outfile_train, index = False)
df_test.to_csv(outfile_test, index = False)

In [21]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,Yr Latest Change,Neighb feat
0,109,533352170,60,RL,4.641784,9.511777,Pave,NoAlley,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,6.587550,GasA,Ex,Y,SBrkr,725,754,0,7.299797,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NoF,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NoPool,NoFence,NoMiscF,0,3,2010,WD,130500,2005.0,cat6
1,544,531379050,60,RL,3.784190,9.349493,Pave,NoAlley,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,6.817831,GasA,Ex,Y,SBrkr,913,1209,0,7.660585,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NoPool,NoFence,NoMiscF,0,4,2009,WD,220000,1997.0,cat4
2,153,535304180,20,RL,4.234107,8.977525,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,6.964136,GasA,TA,Y,SBrkr,1057,0,0,6.964136,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NoF,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NoPool,NoFence,NoMiscF,0,1,2010,WD,109000,2007.0,cat5
3,318,916386060,60,RL,4.304065,9.190444,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,5.953243,GasA,Gd,Y,SBrkr,744,700,0,7.275865,0.0,0.0,2,1,3,1,TA,7,Typ,0,NoF,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NoPool,NoFence,NoMiscF,0,4,2010,WD,174000,2007.0,cat2
4,255,906425045,50,RL,4.418841,9.563529,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,6.517671,GasA,TA,Y,SBrkr,831,614,0,7.276556,0.0,0.0,2,0,3,1,TA,6,Typ,0,NoF,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NoPool,NoFence,NoMiscF,0,3,2010,WD,138500,1993.0,cat4


In [22]:
df_test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Yr Latest Change,Neighb feat
0,2658,902301120,190,RM,1.730200,2.352496,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0.0,Unf,0.0,1020.0,2.026509,GasA,Gd,N,FuseP,908,1020,0,2.116231,0.0,0.0,2,0,4,2,Fa,9,Typ,0,NoF,Detchd,1910.0,Unf,1.0,440.0,Po,Po,Y,0,60,112,0,0,0,NoPool,NoFence,NoMiscF,0,4,2006,WD,1950.0,cat6
1,2718,905108090,90,RL,1.565317,2.336938,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0.0,Unf,0.0,1967.0,2.056407,GasA,TA,Y,SBrkr,1967,0,0,2.158782,0.0,0.0,2,0,6,2,TA,10,Typ,0,NoF,Attchd,1977.0,Fin,2.0,580.0,TA,TA,Y,170,0,0,0,0,0,NoPool,NoFence,NoMiscF,0,8,2006,WD,1977.0,cat6
2,2414,528218130,60,RL,1.655196,2.300335,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554.0,Unf,0.0,100.0,2.074948,GasA,Ex,Y,SBrkr,664,832,0,2.074948,1.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2.0,426.0,TA,TA,Y,100,24,0,0,0,0,NoPool,NoFence,NoMiscF,0,9,2006,New,2006.0,cat4
3,1989,902207150,30,RM,1.668474,2.321450,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,968.0,1.939208,GasA,TA,Y,SBrkr,968,0,0,2.113343,0.0,0.0,1,0,2,1,TA,5,Typ,0,NoF,Detchd,1935.0,Unf,2.0,480.0,Fa,TA,N,0,0,184,0,0,0,NoPool,NoFence,NoMiscF,0,7,2007,WD,2006.0,cat6
4,625,535105100,20,RL,1.689882,2.357407,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609.0,Unf,0.0,785.0,2.017256,GasA,Gd,Y,SBrkr,1394,0,0,2.113427,1.0,0.0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2.0,514.0,TA,TA,Y,0,76,0,0,185,0,NoPool,NoFence,NoMiscF,0,7,2009,WD,1963.0,cat5
